Desafío 2: Geolocalizar eventos en noticias

Utilizando técnicas de tratamiento automático del lenguaje, el objetivo del desafío es estructurar la información de una noticia de la manera siguiente:

- event: principal evento descrito en la noticia
- category: categoria temática del evento
- address: dirección dónde occurió el evento en formato: calle número, comuna, país
- latitud: latitud del evento
- longitud: longitud del evento

Resultado: El formato del archivo CSV output que entregar contiene las columnas siguientes: id_news, event, category, address, latitud, longitud

Evaluación del desafío: Se evaluará su método en base a 100 noticias etiquetadas con las métricas siguientes:
    • Category: Precision y Recall
    • Longitud y Latitud: Exactitud (con una margen de error de 100 metros)

### Primero cargamos las librerias

In [14]:
#!pip install geopy
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


# Load the required libraries
import pandas as pd
import googlemaps
import gmaps 
import geopandas as gpd
import spacy
import random

from googlemaps import Client as GoogleMaps
from keplergl import KeplerGl
from geopy.geocoders import Nominatim
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline
from spacy.training.example import Example
from spacy.util import minibatch
from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL


In [15]:
# Cargar los datos
df = pd.read_csv('dataset_agosto2024.csv')
nlp_loaded = spacy.load("modelo1")#modelo_clasificador_noticias")

In [16]:
geolocator = Nominatim(user_agent="Sophia")

modelAdd="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

tokenizer = AutoTokenizer.from_pretrained(modelAdd)

modelAdd = AutoModelForQuestionAnswering.from_pretrained(modelAdd)

nlp2 = pipeline("question-answering", model=modelAdd, tokenizer=tokenizer)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [17]:
df2 = pd.read_csv('dataset_agosto2024.csv',thousands=',')
df2 = df2.dropna()
df2 = df2[df2['text'] != '(empty)']

questions = [   "¿En el texto, cual es el evento?",
                "¿Cual es el pais del evento?",
                "¿Cual es la comuna del evento?",
                "¿Cual es la calle del evento?"
                ]

df2['event'] = ''
df2['category'] = ''
df2['address'] = ''
df2['latitud'] = ''
df2['longitud'] = ''

df2 = df2.sample(n=100)


In [18]:
for index, row in df2.iterrows():  
    Answer = []
    for question in questions: 
        result = nlp2(question=question, 
                      tokenizer=tokenizer, 
                      model=modelAdd, 
                      context=row['text'])
        
        Answer.append(result['answer'])
    
    #Event
    df2.at[index,'event'] = Answer[0]
    
    #Address
    lugar = Answer[2] +", "+ Answer[1]
    df2.at[index,'address'] = lugar
    
    #Latitude, Longitude
    try:
        location = geolocator.geocode(lugar)

        if location:
            df2.at[index,'latitud'] = location.latitude
            df2.at[index,'longitud'] = location.longitude

        else:
            df2.at[index, 'latitud'] = 0.0
            df2.at[index, 'longitud'] = 0.0

    except Exception as e:
            print(f"Error geocodificando ubicación: {e}")
            df2.at[index, 'latitud'] = 0.0
            df2.at[index, 'longitud'] = 0.0      
    
    #Categoria
    doc = nlp_loaded(row['title'])
    maxscore = 0.0
    maxlabel = ''

    for label, score in doc.cats.items():
        if (score > maxscore):
            maxscore = score
            maxlabel = label

    df2.at[index,'category'] = maxlabel

output_columns = ['id_news', 'event', 'category', 'address', 'latitud', 'longitud']
df2[output_columns].to_csv('tarea1-out.csv', index=False)
print("Creado archivo .csv")

Error geocodificando ubicación: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Per%C3%BA%2C+Per%C3%BA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Creado archivo .csv
